# Deploy a previously created model in SageMaker

Sagemaker decouples model creation/fitting and model deployment. **This short notebook shows how you can deploy a model that you have already created**. It is assumed that you have already created the model and it appears in the `Models` section of the SageMaker console. Obviously, before you deploy a model the model must exist, so please go back and make sure you have already fit/created the model before proceeding. 
For more information about deploying models, see https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-deploy-model.html 

In [12]:
import boto3
from time import gmtime,strftime
import datetime
import sagemaker

sm = boto3.client('sagemaker')

In [24]:
from sagemaker.image_uris import retrieve
training_image = retrieve(framework='xgboost', region='us-east-1', version='latest')
training_image

'811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest'

In [6]:
bucket = "your-bucket-name"
prefix = 'sagemaker/DEMO-hpo-xgboost-dm'
role = sagemaker.get_execution_role()

In [7]:
TrainingJobName = '???' #'name_of_model_from_HPO_analyze_notebook'
model_path = f"s3://{bucket}/{prefix}/output/{TrainingJobName}/output"
model_path

's3://hpo-carrier/sagemaker/DEMO-hpo-xgboost-dm/output'

In [17]:
!aws s3 ls s3://hpo-carrier/sagemaker/DEMO-hpo-xgboost-dm/output/xgboost-tuningjob-15-13-34-13-009-75a46825/output/

2021-03-15 13:46:22       4579 model.tar.gz


In [21]:
def create_model(bucket,  model_name, model_path, container, role):
    try:
        sm.create_model(
            ModelName=model_name,
            PrimaryContainer={
                'Image': container,
                'ModelDataUrl': f'{model_path}/model.tar.gz',
                'Environment': {
                    'SAGEMAKER_REGION': 'us-east-1'
                }
            },
            ExecutionRoleArn=role
        )
    except Exception as e:
        print(f'Error while creating model {model_name}: {e}')
        raise(e)

In [25]:
model_name = 'select a model name'  # ^[a-zA-Z0-9](-*[a-zA-Z0-9])*
create_model(bucket, model_name, model_path, training_image, role)

## Deploy using an inference endpoint

In [26]:
#set endpoint name/config.
endpoint_config_name = 'DEMO-model-config-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_name = 'DEMO-model-config-'  + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [27]:
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])


create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])



Endpoint Config Arn: arn:aws:sagemaker:us-east-1:770499558699:endpoint-config/demo-model-config-2021-03-15-21-35-50
arn:aws:sagemaker:us-east-1:770499558699:endpoint/demo-model-config-2021-03-15-21-35-50
Status: Creating


In [28]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

Status: InService


If you go to the AWS SageMaker service console now, you should see that the endpoint creation is in progress.

## Deploy using a batch transform job

A batch transform job should be used for when you want to create inferences on a dateset and then shut down the resources when inference is finished.  

In [29]:
#config for batch transform
batch_job_name='????' #^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,62}
output_location=f's3://{bucket}/{prefix}/test/pred.csv' #S3 bucket/location
input_location= f's3://{bucket}/{prefix}/test/test.csv'  #S3 bucket/location

In [30]:
request = {
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "TransformOutput": {
        "S3OutputPath": output_location,
        "Accept": "text/csv",
        "AssembleWith": "Line"
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": input_location 
            }
        },
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.m4.xlarge", #change this based on what resources you want to request
            "InstanceCount": 1
    }
}
sm.create_transform_job(**request)

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:770499558699:transform-job/test',
 'ResponseMetadata': {'RequestId': '821a58fb-64fb-43dd-b223-b177cab9d995',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '821a58fb-64fb-43dd-b223-b177cab9d995',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '81',
   'date': 'Mon, 15 Mar 2021 22:09:30 GMT'},
  'RetryAttempts': 0}}